In [ ]:
import cv2 as cv
from ultralytics import YOLO


In [ ]:
video_path='highway.mp4'

In [ ]:
class Annot:
    def __init__(self,img,result,b_color=(0,0,255),b_thickness=1, t_color=(0,255,0),
                 fontFace=cv.FONT_HERSHEY_SIMPLEX,fontScale=0.5,
                 t_thickness=1,lineType=cv.FILLED,bottomLeftOrigin=False):
        
        self.class_names=result.names if len(result.names)>1 else None
        self.img=img
        self.cls=result.boxes.cls.numpy()
        self.prob=result.boxes.conf.numpy()
        self.coord=result.boxes.xyxy.numpy()
        self.center=result.boxes.xywh.numpy()
        self.det_box=len(self.coord)
        self.id=result.boxes.id
        self.b_color=b_color
        self.b_thickness=b_thickness
        self.t_color=t_color
        self.t_thickness=t_thickness
        self.fontFace=fontFace
        self.fontScale=fontScale
        self.lineType=lineType
        self.bottomLeftOrigin=bottomLeftOrigin


        self.objects=[2,3,5,7,]




    def rect(self):
        for i in range(self.det_box):
            #only vehicle
            if self.cls[i] in self.objects:
                start=self.coord[i][:2].astype(int)
                end=self.coord[i][2:].astype(int)
                cv.rectangle(self.img,start,end,self.b_color,self.b_thickness)
    

    def text(self):
        for i in range(self.det_box):
            if self.cls[i] in self.objects and self.id!=None:
                if self.class_names!=None and self.class_names[0]!='0' and self.class_names[0]!=0:
                    text=f'{self.class_names[self.cls[i]]} {(self.prob[i]*100):.0f}% {int(self.id[i])}'
                else:
                    text=f'{(self.prob[i]*100):.0f}%'
                x,y=self.coord[i][:2].astype(int)
                org=(x,y-3)
                cv.putText(img=self.img,text=text,org=org,fontFace=self.fontFace,
                           fontScale=self.fontScale,color=self.t_color,thickness=self.t_thickness,
                           lineType=self.lineType,bottomLeftOrigin=self.bottomLeftOrigin )
    
        
        



In [ ]:
obj_id=set()
left_id=set()
right_id=set()

def count(frame,result, t_color=(0,255,0),
        fontFace=cv.FONT_HERSHEY_SIMPLEX,fontScale=0.5,
        t_thickness=1,lineType=cv.FILLED,bottomLeftOrigin=False):
    global obj_id
    global left_id
    global right_id

    t_color=(0,255,0)
    fontFace=cv.FONT_HERSHEY_SIMPLEX
    fontScale=0.5
    t_thickness=1
    lineType=cv.FILLED
    bottomLeftOrigin=False



    
    center=result.boxes.xywh.numpy()
    id=result.boxes.id

    # x1=0
    # y1=frame.shape[0]
    # x2=frame.shape[1]
    # y2=frame.shape[0]

    height=frame.shape[0]
    width=frame.shape[1]

    #middle hosrizontal
    middle_h=int(height/2)

    #middle vertival
    middle_v=int(width/2)
    
    #horizontal line
    cv.line(frame,(0,middle_h-30),(width,middle_h-30),(0,0,255),1)

    #vertical line
    cv.line(frame,(middle_v,0),(middle_v,height),(0,0,255),1)


    #if object is detected 
    if id!=None:
        for coor,id in zip(center,id):

            #detected objects center coordinate
            obj_x1 = int(coor[0])
            obj_y1 = int(coor[1])
            cv.circle(frame,(obj_x1,obj_y1),2,(0,0,255),4)
            
            #counts all vehicle
            if  (abs(middle_h-obj_y1)<=30) and (0<= obj_x1 <=width):
                obj_id.add(int(id))
            
            #counts all left side vehicle
            if  (abs(middle_h-obj_y1)<=30) and (0<= obj_x1 <=middle_v):
                left_id.add(int(id))
            
            #counts all rigth side vehicle
            if  (abs(middle_h-obj_y1)<=30) and (middle_v<= obj_x1 <=width):
                right_id.add(int(id))

            #write all vehicle number 
            cv.putText(img=frame,text=f'total : {len(obj_id)}',org=(middle_v,50),fontFace=fontFace,
                        fontScale=fontScale,color=t_color,thickness=t_thickness,
                        lineType=lineType,bottomLeftOrigin=bottomLeftOrigin )

            #write left vehicle number 
            cv.putText(img=frame,text=f'left : {len(left_id)}',org=(50,50),fontFace=fontFace,
                        fontScale=fontScale,color=t_color,thickness=t_thickness,
                        lineType=lineType,bottomLeftOrigin=bottomLeftOrigin )

            #write right vehicle number 
            cv.putText(img=frame,text=f'rigth : {len(right_id)}',org=(width-100,50),fontFace=fontFace,
                        fontScale=fontScale,color=t_color,thickness=t_thickness,
                        lineType=lineType,bottomLeftOrigin=bottomLeftOrigin )

In [ ]:

model=YOLO('yolov8m.pt')
video=cv.VideoCapture(video_path)
#video frames number 
length = int(video.get(cv.CAP_PROP_FRAME_COUNT))
frame_proc=0
while True:
    res , frame = video.read()
    results = model.track(frame,stream=True,tracker='botsort.yaml',persist=True,imgsz=640)
    #doesnot work on last frame - freezing and crashing kernel 
    #  if not res:
    #     break
    for i in results:
        annot=Annot(img=frame,result=i)
        annot.rect()
        annot.text()
        count(frame,i)
        cv.imshow('yolo',frame)
    #counts how many frames are processed    
    frame_proc+=1
                                                # if it is last frame close 
    if (cv.waitKey(1) & 0xFF == ord('q')) or frame_proc==length:
        break
video.release()
cv.destroyAllWindows()


    
    
    

In [ ]:
print(len(obj_id))
print(len(left_id))
print(len(right_id))